# Bellman-Ford Algorithm

Given a directed graph $G=(V,E)$ with edges lengths $c_e$ and a source vertex $s$

Compute the length of the shortest $s-v$ path for every other vertex in the graph.

## On Negative Cycles

When a cycle of edges in a graph has a negative total sum, the shortest path can be indefinately decremented by going one more loop around it. 

This invalidates the goal of the algorithm.

On solution is to compute the shortest cycle free path i.e; exclude any cycles. This however, is an NP-hard problem and there are no polynomial algorihtms.

Moving foward, the bellman-ford algorithm will assume that there are no negative cycles. Furthermore, the bellman-ford algorithm is able to quickly check if there is a negative cycle.

## Optimal Substructure

The key idea is to artificially restrict the number of edges in a path. Such that smaller subproblems relate to paths with fewer edges.

Let $G = (V, E)$ be a directed graph with edge lengths $c_e$ and source vertex $s$.

For every $v \in V$ and $i \in \{1, 2, 3, \cdots \}$, define
$$
p_{i,v} = \text{shortest} \; s \rightarrow v \; \text{path with at most} \; i \; \text{edges}
$$

Case 1:

if $p_{i,v}$ has less than $i-1$ edges, then it is the shortest $s \rightarrow v$ path.

Case 2:

if $p_{i,v}$ has $i$ edges. Consider the last hop $w \rightarrow v$. Then $p_{i-1, w}$ is a shortest path.

With these two cases we can develop a recurrence. Let $L_{i, v}$ be the minminum length of a $s \rightarrow v$ path with $\leq i$ edges

For every $v \in V$, $i \in \{1, 2, 3, \cdots\}$
$$
L_{i,v} = \min
\begin{cases}
L_{(i-1), v} \\
\min\limits_{(w, v) \in E}{(L_{(i-1, w)} + c_{(w,v)})}
\end{cases}
$$

Suppose an input graph does not have any negative cycles. Then the shortst paths will not have any cycles, and the longest shortest path will be $\leq (n-1)$ edges. Therefore, we only need to solve subproblems up to $i = n-1$

Therefore the subproblem space is
$$
i \in \{1, 2, \cdots, n-1\} \\
v \in V
$$

## Psuedocode

```
Let A = 2D array

A[0, s] = 0
A[0, v] = +inf for all v != s

for i= 1 to n-1:
    for each v in V:

        A[i, v] = min(
            A[i-1, v]

            min over edges (w, v)(
                A[i-1, w] + c_wv
            )
        )
```

Running time is $O(mn)$. Note that for sparse graphs $m = O(n)$ and for dense graphs $m = O(n^2)$. This aligns with our intution that the running time should be somewhere between $O(n^2)$ and $O(n^3)$

## Checking for Negative Cycles

We claim that after running the Bellman-Ford algorithm for 1 more $n^{th}$ iteration,
$$
G \; \text{has no negative cycles} \iff A[n-1, v] = A[n, v] \quad \forall v \in V
$$

Proof:

The forward direction follows from the proof of correctness of the Bellman-Ford algorithm. 

For the reverse direction, assume
$$
A[n-1, v] = A[n, v] \quad \forall v \in V
$$

Let $d(v)$ denote the common value of $A[n-1, v]$ and $A[n, v]$. This indicates that
$$
d(v) \leq d(w) + c_{w,v} \quad \forall (w, v) \in E
$$

Consider an arbitrary cycle $C$
$$
\sum_{(w,v) \in C}{c_{w, v}} \geq \sum_{(w,v) \in C}{d(v) - d(w)} = 0
$$

Therefore the cycles must have positive overall sum.

## Optimisations

1. Stop early

If at some iteration $j < n-1$
$$
A[j, v] = A[j-1, v] \quad \forall v \in V
$$
Then we can stop the algorithm early as it will only continue to further inherit previous values till completion

2. Space Optimisation

We only need to store the previous array values for the next array. However, in order to maintain information regarding the paths themselves, we will need to store at each vertex a predecessor pointer. Let
$$
B[i, v] = 2^{nd} \; \text{to last vertex on the shortest} \; s \rightarrow v \text{path}
$$
Then, we need only to hop through $B$ values to reconstruct the shortest paths.

To reconstruct a negative cost cycles, we can use depth-first search to check for a cycle of predecessor pointers after each round.


## Creating a routing protocol

1. Switch from source-driven to destination-driven

Every vertex $v$ stores shortest-path distances from $v$ to any relevant destination $t$ and the first hop of a shortest path.

2. Handling Asynchrony

We cannot assume all $A[i-1, v]$ get computed before all $A[i, v]$. We can switch from a "pull-based" to a "push-based" approach. Where each node broadcasts it's connectivity information to all of its neighbours. This works only for static networks and may require up to exponential time for all broadcasts to complete accurately.

3. Handling Failures

In practical applications, links or edges fail all the time. A failure in a link may cause nodes to reference each other in a loop leading to runaway shortest path counts.

To fix this, we can shift from a distance vector protocol to a path vector protocol where each node $v$ maintains the entire shortest path to the destination, not just the next hop.

# All Pairs Shortest Path

Given a directed graph $G=(V,E)$ with edge costs $c_e$ for each edge

Compute the length of a shortest $u \rightarrow v$ path for all pairs of vertices $u, v \in V$ OR report that $G$ contains a negative cycle

## Applying known algorithms

We can use a single-source shortest path subroutine $n$ times.
$$
n\text{-Dijkstra} = O(nm\log{n}) = \begin{cases}
O(n^2\log{n}) & \text{for spare graphs} \\
O(n^3\log{n}) & \text{for dense graphs}
\end{cases} \\[20pt]
n\text{-Bellman-Ford} = O(n^2m) = \begin{cases}
O(n^3) & \text{for spare graphs} \\
O(n^4) & \text{for dense graphs}
\end{cases}
$$

## Floyd-Warshall algorithm

A $O(n^3)$ algorithm that works with negative edge lengths. 

### Optimal Substructure

Impose an arbitrary ordering on the vertices. 

Let $V^{(k)} = \{1, 2, 3, \cdots, k\}$

Suppose $G$ has no negative cycle. Fix a source $i \in V$ and destination $j \in V$ and $k \in \{1, 2, 3, \cdots, k\}$. Let $P=$ shortest cycle free $i \rightarrow j$ path with all internal nodes in $V^{(k)}$

Case 1: if $k$ is not internal to $P$, then $P$ is a shortest cycle free $i \rightarrow j$ path with all internal vertices in $V^{(k-1)}$

Case 2: if $k$ is internal to $P$, then we can split $P$ into two parts $P_1$ and $P_2$ such that 
$$
\fbox{i} \underset{P_1}{\rightarrow} \fbox{k} \underset{P_2}{\rightarrow} \fbox{j}
$$

then,

$P_1=$ shortest cycle free $i \rightarrow k$ path with all internal nodes in $V^{(k-1)}$

$P_2=$ shortest cycle free $k \rightarrow j$ path with all internal nodes in $V^{(k-1)}$

Let $P_{i, j, k}$ be the length of the shortest path from $i$ to $j$ using only intermediate notes from the set $\{1, 2, \cdots, k\}$. Then

$$
P_{i, j, k} = \min\begin{cases}
P_{i, j, k-1} \\
P_{i, k, k-1} + P_{k, j, k-1}
\end{cases}
$$

### Pseudocode

Let A = 3D array indexed by $i, j, k$

Initialise

For all $i, j \in V$:
$$
A_{i, j, 0} = \begin{cases}
0 & \text{if} \; i = j \\
c_{i,j} & \text{if} \; (i,j) \in E \\
+ \infty & \text{if} \; i \neq j \wedge (i,j) \notin E
\end{cases}
$$

```
For k = 1 to n:
    For i = 1 to n:
        For j = 1 to n:

            A[i,j,k] = min(
                A[i, j, k-1],
                A[i, k, k-1] + A[k, j, k-1]
            )
```

### Dealing with negative cycles

We only need to look at the diagonal of the $A$ array to determine if there exists a negative cost cycle. That is
$$
\exists \; A[i,i,n] < 0 \implies \text{negative cost cycle}
$$

Typically, the diagonals should be all $0$ as initialised. Assume there is a negative cost cycle, whose length is $r$. Then at the point at the iteration where $k=r$. For each iteration of $i$ when $j=i$, there will exist a pair of paths leading from $i \rightarrow r$ and $r \rightarrow j$ such that 
$$
A[i, r, r-1] + A[r, i, r-1] \leq 0
$$

This would be less than the base case of $0$, and be placed into the array.

### Reconstructing a shortest path

In addition to $A$, we keep track of 
$$
B[i,j] = \text{max label of an internal node on a shortest} \; i \rightarrow j \; \text{path}
$$

Set $B[i,j] = k$ if the second case of the recurrence is used to compute $A[i,j,k]$

Then to reconstruct,
1. start from $i, j$
2. $k = B[i, j]$
3. Lookup $k_1 = B[i, k]$ and $k_2 = B[k, j]$
4. Repeat

## Johnsons Algorithm

Recall that we can reduce APSP to $n$ incovations of SSSP
- $O(mn\log{n})$ for non negative edge lenghts via Dijkstra
- $O(mn^2)$ for general edge lengths via Bellman-Ford

Johnsons algorithm provides a way to reduce APSP to 1 invocation of Bellman-Ford followed by $n$ inovations fo Dijkstra - reducing the running time to
$$
O(mn) + O(mn\log{n}) = O(mn\log{n})
$$

### Reweighting Transformation

For each edge $e = (u,v) \in E$, let the transformed edge $c^{\prime}_{e}$ be
$$
c^{\prime}_{e} = c_e + p_u - p_v
$$
Where $p_i$ are arbitrary weights assocaited with each node in the graph. 

This acts in a telescoping manner such that for any $s \rightarrow t$ path, the length of the path $L$ is mapped to
$$
L \rightarrow L + p_s - p_t
$$
This transformation therefore preserves the minimum edge length property.

We aim to assign the values of $p_i$ such that all the edge lengths are nonnegative

### Finding the Weights

1. Add a new vertex $s$ to the graph. Attach edges from $s \rightarrow i$ for every vertex $i \in V$ with cost $0$.
2. Use Bellman-Ford to compute the shortest path distances from $s$ to each vertex.
3. Assign these negative shortest distances to each vertex as the weights $p_i$

### Proof that weights will be non negative

Fix an edge $e = (u, v)$. The weights $p_u$, $p_v$ are the shortest path distances from $s$ to $u$ and $v$ respectively.

Since $p_v$ is the shortest path from $s$ to $v$, it is neccesarily the case that
$$
p_v \leq p_u + c_e
$$

Therefore
$$
0 \leq p_u + c_e - p_v = c^{\prime}_e
$$

As required

### Pseudocode

1. Form $G^{\prime}$ by adding a new vertex $s$ and a new edge $(s,v)$ with length 0 for each edge $v \in G$
2. Run Bellman-Ford on $G^\prime$ with source $s$.
    - If Bellman-Ford finds a negative cost cycle, halt and return
3. For each vertex $v \in G$ define $p_v =$ length of a shortest $s \rightarrow v$ path in $G^{\prime}$. For each edge $c_e$ define
$$
c^{\prime}_{e} = c_e + p_u - p_v
$$
4. For each vertex $u \in G$, Run Dijkstra's algorithm with the updated edge costs to compute the shortest path distance $d^{\prime}(u,v)$ for each $v \in G$
5. Revise the path lengthts for each $u \rightarrow v$ paths
$$
d(u,v) = d^{\prime}(u,v) - p_u + p_v
$$

# Problem Set

## 2

Consider the following optimization to the Bellman-Ford algorithm. 

Given a graph $G=(V,E)$ with real-valued edge lengths, we label the vertices $V=\{1,2,3,…,n\}$. The source vertex sss should be labeled "1", but the rest of the labeling can be arbitrary. 

Call an edge $(u,v) \in E$ forward if $u \lt v$ and backward if $u \gt v$.  

In every odd iteration of the outer loop (i.e., when $i=1,3,5,\cdots$), we visit the vertices in the order from $1$ to $n$.

In every even iteration of the outer loop (when $i=2,4,6,\cdots$), we visit the vertices in the order from $n$ to $1$.

In every odd iteration, we update the value of $A[i,v]$ using only the forward edges of the form $(w,v)$, using the most recent subproblem value for $w$ (that from the current iteration rather than the previous one).That is, we compute 
$$
A[i,v]=\min\begin{cases}⁡
    A[i−1,v] \\
    \min_{⁡(w,v)} A[i,w]+c_wv
\end{cases}
$$

where the inner minimum ranges only over forward edges sticking into $v$.

Note that all relevant subproblems from the current round are avaliable for constant time lookup. In even iterations, we compute this same recurrence using only the backward edges. 

It correctly computes the shortest paths iff the input graph has no negative cost cycle.

Indeed.  Can you prove it?  As a preliminary step, prove that with a directed acyclic graph, considering destinations in topological order allows one to compute correct shortest paths in one pass (and thus, in linear time).  Roughly, pass $i$ of this optimized Bellman-Ford algorithm computes shortest paths amongst those comprising at most $i$ "alternations" between forward and backward edges.

# Optional Theory Problem

## 1

Recall the asynchronous version of the Bellman-Ford algorithm discussed in the internet routing section of Bellman-Ford. Prove that, in the worse case, this algorithm requires an exponential number of iterations to converge.

# Programming Assingment

# 1 

We will implement the algorithms to solve the All Pairs Shortest-Path problem.

For each file 'g* Week 1.txt' the first line indicates the number of vertices and edges. Each subsequent line describes an edge with the shape

[tail] [head] [cost]

The task is to compute the shortest shortest path.

Identify if any of the three graphs have no negative cycles. For each graph, compute all pairs shortest paths and remember the smallest one.

If each of the three graphs has a negative cost cycle enter "NULL" as the answer. If exactly one graph has no negative cost cycles, enter the length of its shortest shortest path. If two graphs have no negative cost cycles, enter the smallest of the two graphs.

For fun, try computing the shortest shortest path of the large graph 'gLarge Week 1.txt'

In [2]:
from typing import Literal, Tuple

def gen_fname(number: Literal[1, 2, 3]):
    return f'g{number} Week 1.txt'

def load_data(file: str):
    with open(file) as f:
        num_nodes, num_edges = [int(x) for x in next(f).split(" ")]
        
        g: dict[int, list[Tuple[int, int]]] = {tail: [] for tail in range(1, num_nodes+1)}
        g_rv: dict[int, list[Tuple[int, int]]] = {head: [] for head in range(1, num_nodes+1)}
        
        for line in f:
            tail, head, cost = [int(x) for x in line.split(" ")]
            
            g[tail].append((head, cost))
            g_rv[head].append((tail, cost))
        
        return num_nodes, num_edges, g, g_rv

In [38]:
import numpy as np
from local_utils import Heap

def johnsons(num_nodes, g: dict[int, list[Tuple[int, int]]], g_rv: dict[int, list[Tuple[int, int]]]):

    # Bellman Ford Part

    # augment with a new node 
    probe_node = 0
    g_rv[probe_node] = []
    
    # array containing the edge costs from probe_node to all other nodes
    left = [0 for _ in range(0, num_nodes +1)]
    
    for i in range(1, num_nodes +1):
        right = []
        early_stop = True

        for v in range(0, num_nodes +1):
            
            if len(g_rv[v]) == 0:
                right.append(left[v])
            else:
                right.append(min(
                    left[v],
                    min([left[tail] + cost for (tail, cost) in g_rv[v]])
                ))
            
            if right[v] != left[v]:
                early_stop = False
        
        left = right
        
        # negative cylce check 
        if i == num_nodes and not early_stop:
            # halt and return there is a neg cycle
            return False
        
        # early stop
        if early_stop:
            break
                
    node_weights = left
    print("Weights Calculated")

    # n-Dijkstra part
    global_closest_pair = None
    for startNode in range(1, num_nodes +1):
        print(startNode, end="\r")
        
        nodeHeap = Heap[int](type="min")
        costs = {node: np.Inf for node in range(1, num_nodes +1)}
        
        for head, cost in g[startNode]:
            costs[head] = cost + node_weights[startNode] - node_weights[head]
        
        nodeHeap.build_heap(
            list(costs.keys()),
            costs
        )
        
        closest_node = None
        while nodeHeap.size > 0:
            greedy_node, greedy_score = nodeHeap.extract_lead()

            corrected_score = greedy_score - node_weights[startNode] + node_weights[greedy_node]

            if closest_node is None or corrected_score < closest_node[1]:
                closest_node = (greedy_node, corrected_score)
            
            for head, cost in g[greedy_node]:
                if head in nodeHeap:
                    old_key = nodeHeap.keys[head]
                    new_key = greedy_score + cost + node_weights[greedy_node] - node_weights[head]
                    
                    if new_key < old_key:
                        nodeHeap.rekey(head, new_key)
                    
        if global_closest_pair is None or closest_node[1] < global_closest_pair[1]:
            global_closest_pair = (
                (startNode, closest_node[0]),
                closest_node[1]
            )
    
    return global_closest_pair

In [39]:
num_nodes, num_edges, g, g_rv = load_data(gen_fname(3))
johnsons(num_nodes, g, g_rv)

Weights Calculated


((399, 904), -19)

In [40]:
import numpy as np

def floydWarshall(num_nodes, g: dict[int, list[Tuple[int, int]]]):
    arr_shape = (num_nodes +1, num_nodes +1)
    left_k = np.full(arr_shape, np.Inf)
    np.fill_diagonal(left_k, 0)
    
    for tail, edges in g.items():
        for head, cost in edges:
            left_k[tail, head] = cost

    for k in range(1, num_nodes +1):
        print(k, end="\r")
        # sped up using numpy!!
        right_k = np.fromfunction(
            lambda i, j: np.minimum(left_k[i, j], left_k[i, k] + left_k[k, j]),
            arr_shape,
            dtype=int
        )
        left_k = right_k

    # negative cycle check
    if np.any(left_k.diagonal() < 0):
        return False
    
    idx = np.unravel_index(left_k.argmin(), arr_shape)
    return (idx, left_k[idx])

In [41]:
num_nodes, num_edges, g, g_rv = load_data(gen_fname(3))
floydWarshall(num_nodes, g)

((399, 904), -19.0)